In [21]:
import pandas as pd

landDF = pd.read_csv('../0703/test_data/all_test_lat.csv', index_col=False)
# 找lat, lng缺的地址
missing_lat_list = landDF[landDF['lat'].isnull()].head().index.tolist()
unique_missing = landDF[landDF['lat'].isnull()]['土地區段位置/建物區段門牌'].unique()

In [13]:
landDF[landDF['lat'].isnull()].head().index.tolist()

[2, 3, 7, 14, 18]

In [17]:
len(landDF[landDF['lat'].isnull()].index.tolist())

7829

In [20]:
landDF[landDF['lat'].isnull()]['土地區段位置/建物區段門牌'].unique()

array(['新北市板橋區文化路二段571~600號', '新北市土城區青雲路281巷1~30號',
       '新北市板橋區南雅西路二段301巷31~60號', ..., '御史段391~420地號', '五股坑一段1471~1500地號',
       '新北市板橋區忠孝路38巷1~30號'], dtype=object)

In [23]:
len(unique_missing)

3852

In [37]:
import googlemaps
import os
from os.path import join, dirname
from dotenv import load_dotenv, find_dotenv
dotenv_path = join(dirname('__file__'), '.env')
load_dotenv(dotenv_path, override=True)  # 設定 override 才會更新變數哦！
GOOGLE_PLACES_API_KEY = os.environ.get("GOOGLE_PLACES_API_KEY1")

# Client
gmaps = googlemaps.Client(key=GOOGLE_PLACES_API_KEY)

In [38]:
geoDF = pd.DataFrame()

In [35]:
for i in range(0, 5):
    if not '地號' in unique_missing[i]:
        print('地號' in unique_missing[i])

False
False
False
False


In [39]:
for i in range(0, len(unique_missing)):
    # 確保不會查到土地的地址 '地號'
    if not '地號' in unique_missing[i]:
        geo_result = gmaps.geocode(unique_missing[i])
        try:
            # print(landDF['土地區段位置/建物區段門牌'].unique()[i], geo_result[0]['geometry']['location']['lat'], geo_result[0]['geometry']['location']['lng'])
            geoSeries = pd.DataFrame({'address': [unique_missing[i]], 'lat': [geo_result[0]['geometry']['location']['lat']], 'lng': [geo_result[0]['geometry']['location']['lng']]})
            geoDF = geoDF.append(geoSeries, ignore_index=True)
        except:
            continue

geoDF.to_csv('../0703/test_data/geo_info_108_test.csv', index=False, encoding='utf_8_sig')